In [9]:
# Bitcoin Analysis and Visualization Script - Version 013
# This version includes:
# - Bar charts for metrics visualization with price on left y-axis and indicators on right y-axis
# - Heatmap for overall sentiment analysis with improved styling
# - Removal of grid lines from all charts
# - Adjusted and aligned heatmap color legend
# - Saving all charts to a single HTML file with white background

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2020, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True):
    price_change = df[column].diff()
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)
    avg_gains = gains.rolling(window=rsi_window, min_periods=1).mean()
    avg_losses = losses.rolling(window=rsi_window, min_periods=1).mean()
    relative_strength = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        rsi_min = rsi.rolling(window=window_norm, min_periods=1).min()
        rsi_max = rsi.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

def calculate_spot_cvd_bias(df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True):
    rolling_sum = df[column].rolling(window=window_sum).sum()
    
    if normalize:
        rolling_min = rolling_sum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = rolling_sum.rolling(window=window_norm, min_periods=1).max()
        normalized_bias = 2 * (rolling_sum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_bias
    else:
        return rolling_sum

def calculate_spot_volume_momentum(df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True):
    fast_ma = df[column].rolling(window=fast_window).mean()
    slow_ma = df[column].rolling(window=slow_window).mean()
    volume_momentum = fast_ma / slow_ma

    if normalize:
        rolling_min = volume_momentum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = volume_momentum.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (volume_momentum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_momentum
    else:
        return volume_momentum

In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def aggregate_indicators(df, columns, method='equal_weight'):
    """
    Aggregate financial behavior indicators from a DataFrame, excluding initial rows with NaN values.
    
    :param df: pandas DataFrame containing the indicators
    :param columns: list of column names to be aggregated
    :param method: 'equal_weight' or 'PCA'
    :return: pandas Series with the aggregated indicator
    """
    # Ensure all specified columns exist in the DataFrame
    if not all(col in df.columns for col in columns):
        raise ValueError("Some specified columns are not in the DataFrame")
    
    # Extract the relevant columns
    data = df[columns]
    
    if method == 'equal_weight':
        # For equal weight, we can use pandas mean which automatically skips NaN
        weight = 1 / len(columns)
        print(f"weights are all equal = {weight:.4f}")
        return data.mean(axis=1)
    
    elif method == 'PCA':
        # Remove rows with any NaN values
        data_clean = data.dropna()
        
        if len(data_clean) == 0:
            raise ValueError("No complete rows found after removing NaN values")
        
        # Standardize the data
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_clean)
        
        # Perform PCA
        pca = PCA(n_components=1)
        pca_result = pca.fit_transform(data_scaled)
        
        # Calculate weights from the first principal component
        weights = pca.components_[0] / np.sum(np.abs(pca.components_[0]))
        
        print("PCA weights:")
        for col, weight in zip(columns, weights):
            print(f"{col}: {weight:.4f}")
        
        # Calculate weighted sum for all rows, including those with NaN
        weighted_sum = data.mul(weights).sum(axis=1)
        
        # Normalize the weighted sum to be in the same range as input data
        min_val, max_val = data.min().min(), data.max().max()
        normalized_sum = (weighted_sum - weighted_sum.min()) / (weighted_sum.max() - weighted_sum.min())
        normalized_sum = normalized_sum * (max_val - min_val) + min_val
        
        return normalized_sum
    
    else:
        raise ValueError("Invalid method. Choose 'equal_weight' or 'PCA'")

In [11]:
# Apply the functions to our merged_df
merged_df['Price Momentum'] = calculate_momentum_rsi(merged_df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True)
merged_df['Spot CVD Bias'] = calculate_spot_cvd_bias(merged_df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True)
merged_df['Spot Volume Momentum'] = calculate_spot_volume_momentum(merged_df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True)

In [12]:
merged_df['aggregate_indicator_equalweight']= aggregate_indicators(merged_df, columns=['Price Momentum','Spot CVD Bias','Spot Volume Momentum'], method='equal_weight')
merged_df['aggregate_indicator_PCA']= aggregate_indicators(merged_df, columns=['Price Momentum','Spot CVD Bias','Spot Volume Momentum'], method='PCA')

weights are all equal = 0.3333
PCA weights:
Price Momentum: 0.4986
Spot CVD Bias: 0.4992
Spot Volume Momentum: -0.0022


In [13]:
def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=365)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define colors
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
    RED_COLOR = 'red'
    GREEN_COLOR = 'green'

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    
    # Add red color for positive values (swapped)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator > 0, 0),
            name=f"{indicator_column} (Positive)",
            line=dict(color=RED_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add green color for negative values (swapped)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color=GREEN_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 3, 5, 7, 9, 11]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]

    # Determine the color based on the last value (swapped)
    indicator_color = RED_COLOR if last_value >= 0 else GREEN_COLOR

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title={
            'text': "Date",
            'font': {'color': 'black', 'size': 18}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [14]:

# Create charts for each indicator
indicators = [
    ('Price Momentum', "Bitcoin: Price Momentum"),
    ('Spot CVD Bias', "Bitcoin: Spot CVD Bias"),
    ('Spot Volume Momentum', "Bitcoin: Spot Volume Momentum")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_last_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [15]:
merged_df.tail()

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum,Spot CVD Bias,Spot Volume Momentum,aggregate_indicator_equalweight,aggregate_indicator_PCA
t,,,,,,,,
2024-09-27,65767.337050,9.809854e+07,9.048639e+09,0.481533,0.540986,-0.645320,0.125733,0.512141
2024-09-28,65920.337168,-1.291324e+07,3.348024e+09,0.591867,0.542171,-0.649086,0.161651,0.567773
2024-09-29,65673.004639,-1.723601e+07,3.262842e+09,0.686754,0.518255,-0.704186,0.166941,0.603279
2024-09-30,63245.674886,-8.859052e+07,1.020997e+10,0.451849,0.355211,-0.716161,0.030300,0.404721
2024-10-01,60895.040343,-1.210706e+08,1.326828e+10,-0.013276,0.366368,-0.738864,-0.128591,0.178350


In [16]:
merged_df.to_csv('bitcoin_analysis_data_spot.csv', index=True)

In [17]:
def chart_aggregate(df, indicator_col, price_col, chart_title="Price and Indicator Chart"):
    # Filter for the last 2 years of data
    last_2_years = df.index.max() - pd.DateOffset(years=1)
    df_last_2_years = df[df.index >= last_2_years]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the scatter plot
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_last_2_years.index,
            y=df_last_2_years[price_col],
            mode='markers',
            marker=dict(
                color=df_last_2_years[indicator_col],
                size=8,
                line=dict(width=0.5, color='black'),  # Thinner black border for dots
                colorscale='RdYlGn',  # Red-Yellow-Green colorscale
                colorbar=dict(
                    title='Indicator',
                    tickvals=[-1, 0, 1],
                    ticktext=['-1', '0', '1'],
                    lenmode='fraction',
                    len=0.75,
                    x=1.02  # Move colorbar slightly to the right
                ),
                cmin=-1,
                cmax=1,
            ),
            text=[f"Date: {date.strftime('%Y-%m-%d')}<br>Price: {price:.2f}<br>Indicator: {ind:.2f}" 
                  for date, price, ind in zip(df_last_2_years.index, df_last_2_years[price_col], df_last_2_years[indicator_col])],
            hoverinfo='text'
        )
    )

    # Add vertical lines for every 3 months (January, April, July, October)
    start_date = df_last_2_years.index[0].to_period('Q').to_timestamp()
    end_date = df_last_2_years.index[-1].to_period('Q').to_timestamp()
    for date in pd.date_range(start=start_date, end=end_date, freq='Q'):
        if df_last_2_years.index[0] <= date <= df_last_2_years.index[-1]:
            fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=1, opacity=0.7)

    # Get the last value of the indicator and price
    last_value = df_last_2_years[indicator_col].iloc[-1]
    last_price = df_last_2_years[price_col].iloc[-1]

    # Add annotation for the last value
    fig.add_annotation(
        x=df_last_2_years.index[-1],  # Fixed x position at the last date
        y=last_price,  # y position based on the last price
        text=f"{last_value:.2f}",  # Simplified annotation
        showarrow=False,
        font=dict(size=14, color='green' if last_value > 0 else 'red'),
        align="left",
        xanchor="left",
        yanchor="middle",
        xshift=10,  # Shift the annotation slightly to the right of the last point
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,
        hovermode="closest",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,
        height=450,
        margin=dict(r=80)  # Increase right margin to accommodate colorbar
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 12},
        zeroline=False,
        dtick="M3",  # Show tick every 3 months
        tickformat="%b %Y",  # Format as "Mon Year" for every tick
        showline=False,  # Remove x-axis line
        mirror=False
    )
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 12},
        zeroline=False,
        showline=True,
        linewidth=1,
        linecolor=GREY_COLOR,
        mirror=False,
        ticks='outside',  # Show ticks outside
        ticklen=5,  # Length of the ticks
        tickcolor=GREY_COLOR  # Color of the ticks
    )

    return fig

# Example usage
fig2 = chart_aggregate(merged_df, 'aggregate_indicator_equalweight', 'price_usd_close', "Spot Sentiment Aggregated Indicator")
fig2.show()
pio.write_html(fig2, file=f'bitcoin_analysis_spot_aggr_last_1year.html')
print("All charts have been displayed and saved as separate HTML files.")

/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_2579/3971465082.py:42: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



All charts have been displayed and saved as separate HTML files.
